<a href="https://colab.research.google.com/github/codingFerryman/crypto_market_hierarchy_structure/blob/main/src/crypto_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from scipy.signal import correlate
from scipy.stats import spearmanr
from scipy.stats import pearsonr
import glob
import os
from pathlib import Path
from utils import check_integrity, datestring_to_timestamp, load_data

###Preprocessing and Load data

+ Preprocessing missing value

+ Load data

+ Build up list of original data vectors

+ Get the name list of cryptocurrency

#### Load data, Build up list of original data vectors, Get the name list of cryptocurrency

In [37]:
# Get file paths
def get_file_path(folder_path):
  paths = glob.glob(os.path.join(folder_path, "*"))
  paths = sorted(paths)
  return paths

In [53]:
# create the list of crytocurrencies price records, 
# each item means the list of original price vectors of each cryptocurrency
# each vector contains all the close prices of per day / per week (determing with time scale)
def get_interval(timescale):
  if timescale == '3h':
    return 8
  if timescale == '30m':
    return 48
  if timescale == '6h':
    return 4
  else:
    return int(timescale)

def get_days(timescale):
  if timescale == '1d':
    return 1
  if timescale =='1w':
    return 7

def build_data_list(paths, interval, timescale, start_from_timestamp, end_before_timestamp):
  #interval = get_interval(interval)
  timescale = get_days(timescale)
  data_list = []
  name_list = []
  
  for i in range(0, len(paths)):
    idx = 0
    shift_idx = interval*timescale - 1
    cryptocurrency_price_list = []
    
    cryptocurrency_df = load_data(start_from_timestamp, end_before_timestamp, paths[i], fill_na=True)
    
    #record the name of crytocurrency

    name_list.append(cryptocurrency_df.index[0][1])

    #discard other columns except close price
    close_price_df = cryptocurrency_df["close"]
    #print(close_price_df.head(10))
  
    while shift_idx <= len(close_price_df):
      vector = close_price_df.iloc[idx:shift_idx + 1]
      #print(len(vector))
      #Calculate the price ratio of changing
      vector = vector.to_frame()
      vector = vector.pct_change().fillna(0)

      cryptocurrency_price_list.append(np.asarray(vector))
      idx += interval*timescale
      shift_idx += interval*timescale

    #print(len(cryptocurrency_price_list))
    data_list.append(cryptocurrency_price_list)  

  return data_list, name_list


In [44]:
def build_data_list_once(paths, interval, timescale, start_from_timestamp, end_before_timestamp):
  interval = get_interval(interval)
  timescale = get_days(timescale)
  
  name_list = []
  cryptocurrency_price_list = []
  for i in range(0, len(paths)):
    idx = 0
    shift_idx = interval*timescale - 1
    
    
    cryptocurrency_df = load_data(start_from_timestamp, end_before_timestamp, paths[i], fill_na=True)
    
    #record the name of crytocurrency

    name_list.append(cryptocurrency_df.index[0][1])

    #discard other columns except close price
    close_price_df = cryptocurrency_df["close"]
  
    vector = close_price_df.iloc[idx:shift_idx + 1]
  
    #Calculate the price ratio of changing
    vector = vector.to_frame()
    vector = vector.pct_change().fillna(0)
   
    flat_vector = vector['close'].values
    #print(flat_vector)
    cryptocurrency_price_list.append(np.asarray(flat_vector)) 

  return cryptocurrency_price_list, name_list

###Define functions to calculate correlation matrix

+ Pearson correlation coefficient
+ Spearman rank-order correlation coefficient

In [4]:
# Calculate correlation matrix - Pearson correlation coefficient
def cm_pearson(x, y):
  pccs = pearsonr(x,y)
  return pccs[0]

# Calculate correlation matrix - Spearman rank-order correlation coefficient
def cm_spearman(x, y):
  sccs = spearmanr(x, y)
  return sccs[0]

### Calculate Correlation matrix

In [5]:
def calculate_cm(data_list, func):
    N = len(data_list)
    cm_res_list = []
    cm_res = np.zeros((N,N))
    for day in range(0, len(data_list[0])):
      for i in range(N):
        for j in range(N):
          data_list[i][day] = data_list[i][day].squeeze()
          data_list[j][day] = data_list[j][day].squeeze()
          res = func(data_list[i][day], data_list[j][day]) 
          cm_res[i,j] = res
      
      cm_res_list.append(cm_res)
    
    return cm_res_list

In [13]:
def calculate_cm_once(data_list, func):
    N = len(data_list)
    cm_res = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            res = func(data_list[i], data_list[j]) 
            cm_res[i,j] = res
    #print(cm_res.shape)        
    return cm_res

In [9]:
# test code

data_path = "../data"
start_from = "2021-04-12"
end_before = "2021-06-15"
interval = "3h"

start_from_timestamp = datestring_to_timestamp(start_from)
end_before_timestamp = datestring_to_timestamp(end_before)
data_3h_path_list = list(Path(data_path, interval).iterdir())
#print(len(data_3h_path_list))
coins_1D_pass = []
for file_path in data_3h_path_list:
    _coin_code = file_path.name.split("_")[0]#
    try:  
        if check_integrity(start_from, end_before, file_path) is not None:
            coins_1D_pass.append(file_path)
    except KeyError: print(file_path)
    

#print(len(coins_1D_pass))
# load csv and build list
crypto_data_list, name_list = build_data_list(coins_1D_pass, '6h', '1w',start_from_timestamp, end_before_timestamp)

# calculate correlation matrix

correlation_matrix_pearson = calculate_cm(crypto_data_list, cm_pearson)
#print(len(crypto_data_list))
print(len(correlation_matrix_pearson))
print(correlation_matrix_pearson[0])


18
[[ 1.          0.76910766  0.91588619 ... -0.03218666  0.28775571
   0.86414338]
 [ 0.76910766  1.          0.81287892 ...  0.15680194  0.33245241
   0.83994779]
 [ 0.91588619  0.81287892  1.         ...  0.05677791  0.26981957
   0.87682151]
 ...
 [-0.03218666  0.15680194  0.05677791 ...  1.          0.35113036
   0.18738362]
 [ 0.28775571  0.33245241  0.26981957 ...  0.35113036  1.
   0.30387049]
 [ 0.86414338  0.83994779  0.87682151 ...  0.18738362  0.30387049
   1.        ]]


### Save correlation matrix to files in format of Numpy array

In [116]:
np.save('0412_0615_1W_6h_69.npy', correlation_matrix_pearson, allow_pickle=True)

In [117]:
# save namelist
print(name_list)
textfile = open("0412-0615_1W_6h_namelist.txt", "w")
for element in name_list:
    textfile.write(element + "\n")
textfile.close()

['VSY', 'XAUT', 'CHZ', 'DAI', 'YFI', 'VET', 'DOT', 'BAL', 'BSV', 'BOSON', 'NEO', 'SUN', 'EGLD', 'BTC', 'ETC', 'ESS', 'ETP', 'RBT', 'BCHN', 'XSN', 'UOP', 'LINK', 'CLO', 'ZRX', 'SOL', 'ANT', 'AVAX', 'MLN', 'EOS', 'IQX', 'FIL', 'OMG', 'FET', 'XRP', 'XDC', 'XVG', 'SNX', 'PLU', 'ENJ', 'KSM', 'QTM', 'LTC', 'AAVE', 'KNC', 'BTT', 'XTZ', 'ZIL', 'COMP', 'CEL', 'BTG', 'SUSHI', 'LEO', 'BAT', 'HEZ', 'XMR', 'JST', 'UOS', 'WBT', 'ZEC', 'XLM', 'XRA', 'ETH', 'MKR', 'BAND', 'GTX', 'UNI', 'TRX', 'SAN', 'ADA']


In [ ]:
# calculate correlation and check integrity by timescale

# timescale: daily
data_path = "../data"

# datarange
date_range = pd.date_range(start="2021-04-12",end="2021-06-15", freq='D').strftime('%Y-%m-%d')
#print(len(date_range))
interval = '30m'

cm_list = []
name_lists = []

for i in range(0, len(date_range)-1):
    start_from = date_range[i]
    end_before = date_range[i + 1]
  
    start_from_timestamp = datestring_to_timestamp(start_from)
    end_before_timestamp = datestring_to_timestamp(end_before)

    data_list = list(Path(data_path, interval).iterdir())

    checked_list = []
    for file_path in data_list:
        if check_integrity(start_from, end_before, file_path) is not None:
            checked_list.append(file_path)

    # load csv and build list
    crypto_daily_list, name_list = build_data_list_once(checked_list, '30m', '1d',start_from_timestamp, end_before_timestamp)

    # calculate correlation matrix
    correlation_matrix_pearson = calculate_cm_once(crypto_daily_list, cm_pearson)
    print(correlation_matrix_pearson.shape)

    cm_list.append(correlation_matrix_pearson)
    name_lists.append(name_list)
    


In [ ]:
print(len(cm_list))
for item in cm_list:
    print(item.shape)

In [ ]:
print(name_lists)
for item in name_lists:
    print(len(item))

In [51]:
#save to .npy file
np.save('../cm_data/0412_0615_1d_30m.npy', cm_list, allow_pickle=True)

In [52]:
# save namelist
import json

with open('../cm_data/0412-0615_1d_30m_namelist.json', 'w') as f:
    json.dump(name_lists, f)

In [ ]:
# time: weekly

data_path = "../data"

# datarange
date_range = pd.date_range(start="2021-04-12",end="2021-06-15s", freq='7D').strftime('%Y-%m-%d')
print(date_range)
interval = '3h'

cm_list = []
name_lists = []

for i in range(0, len(date_range)-1):
    start_from = date_range[i]
    end_before = date_range[i + 1]
  
    start_from_timestamp = datestring_to_timestamp(start_from)
    end_before_timestamp = datestring_to_timestamp(end_before)

    data_list = list(Path(data_path, interval).iterdir())
    #print(len(data_list))
    checked_list = []
    for file_path in data_list:
        if check_integrity(start_from, end_before, file_path) is not None:
            checked_list.append(file_path)
    #print(len(checked_list))
    
   
    # load csv and build list
    crypto_daily_list, name_list = build_data_list_once(checked_list, '3h', '1w',start_from_timestamp, end_before_timestamp)
    print(len(crypto_daily_list))
    print(crypto_daily_list[0].shape)
    
    
    # calculate correlation matrix
    correlation_matrix_pearson = calculate_cm_once(crypto_daily_list, cm_pearson)
    print(correlation_matrix_pearson.shape)
    print("-----------------")
    
    cm_list.append(correlation_matrix_pearson)
    name_lists.append(name_list)


In [ ]:
print(len(cm_list))
for item in cm_list:
    print(item.shape)

In [ ]:
print(len(name_lists))
for item in name_lists:
    print(len(item))

In [26]:
#save to .npy file
np.save('../cm_data/0412_0615_1W_3h.npy', cm_list, allow_pickle=True)

/opt/homebrew/Caskroom/miniconda/base/envs/outliers-vis/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [27]:
# save namelist

with open('../cm_data/0412-0615_1W_3h_namelist.json', 'w') as f:
    json.dump(name_lists, f)

In [ ]:
print(len(name_lists))
for i in range(0, len(name_lists)):
    print(len(name_lists[i]))

In [ ]:
# monthly

data_path = "../data"

# datarange
date_range = ['2020-12-01','2020-12-31', '2021-01-01','2021-01-31', '2021-02-01','2021-02-28', '2021-03-01','2021-03-31', '2021-04-01','2021-04-30',
       '2021-05-01','2021-05-31', '2021-06-01','2021-06-30', '2021-07-01','2021-07-31', '2021-08-01','2021-08-31', '2021-09-01','2021-09-30',
       '2021-10-01','2021-10-31', '2021-11-01','2021-11-30']
print(len(date_range))
interval = '1D'

cm_list = []
name_lists = []
interval = [31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30]
j = 0
for i in range(0, 12):
    start_from = date_range[j]
    end_before = date_range[j + 1]
  
    start_from_timestamp = datestring_to_timestamp(start_from)
    end_before_timestamp = datestring_to_timestamp(end_before)

    data_list = get_file_path('../data/1D')
    #print(len(data_list))
    checked_list = []
    for file_path in data_list:
        if check_integrity(start_from, end_before, file_path) is not None:
            checked_list.append(file_path)
    #print(len(checked_list))
    
   
    # load csv and build list
    crypto_daily_list, name_list = build_data_list_once(checked_list, interval[i], '1d',start_from_timestamp, end_before_timestamp)
    print(len(crypto_daily_list))
    print(crypto_daily_list[0].shape)
    
    
    # calculate correlation matrix
    correlation_matrix_pearson = calculate_cm_once(crypto_daily_list, cm_pearson)
    print(correlation_matrix_pearson.shape)
    print("-----------------")
    
    cm_list.append(correlation_matrix_pearson)
    name_lists.append(name_list)

    j = j + 2

In [41]:
#save to .npy file
np.save('../cm_data/12_11_M_1d.npy', cm_list, allow_pickle=True)

/opt/homebrew/Caskroom/miniconda/base/envs/outliers-vis/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [42]:
# save namelist

with open('../cm_data/12_11_M_1d_namelist.json', 'w') as f:
    json.dump(name_lists, f)

In [ ]:
print(len(name_lists))
for item in name_lists:
    print(len(item))